# Apex Tracking - Live Demo

In this notebook we'll used the trained model to steer the robot around the course.

First, let's load the model we trained.

In [1]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('throttle_model.pth'))

device = torch.device('cuda')
model = model.to(device).eval().half()

Next, we'll optimize the model using ``torch2trt``.  To do this, we feed example data as input.  We need to call ``model.eval`` before converting with torch2trt.  We'll enable ``fp16_mode`` which allow network optimizations to use reduced precision operations internally.

This may take a couple minutes...

In [2]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

We can save the model by calling the following.  This way we can re-use the optimized model without having to run the optimization procedure again.

In [3]:
torch.save(model_trt.state_dict(), 'throttle_model_trt_half.pth')

In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('throttle_model_trt_half.pth'))

We can save the optimized model by loading it into a TRTModule as follows.

In [13]:
import torch
from torch2trt import TRTModule

apex_model_trt = TRTModule()
apex_model_trt.load_state_dict(torch.load('apex_model_trt_half.pth'))

Next, let's create the racecar instance which we'll use to control the robot. 

In [4]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar(
    steering_gain=1.0,
    steering_offset=0.0,
    throttle_gain=-1.0,
    throttle_offset=0.0
)

In [20]:
car.throttle_gain=1.0
car.steering_gain=-1.0

And some sliders to tune parameters used for steering the robot.

In [23]:
from IPython.display import display
import ipywidgets

steering_gain = ipywidgets.FloatSlider(min=0.0, max=3.0, value=0.7, step=0.001, description='steering gain')
steering_offset = ipywidgets.FloatSlider(min=-0.5, max=0.5, value=0.0, step=0.001, description='steering offset')
speed_gain = ipywidgets.FloatSlider(min=0.0, max=1.5, value=0.0, step=0.001, description='speed gain')
speed_offset = ipywidgets.FloatSlider(min=-1.0, max=1.0, value=0.0, step=0.001, description='speed offset')

display(
    steering_gain,
    steering_offset,
    speed_gain,
    speed_offset
)

FloatSlider(value=0.7, description='steering gain', max=3.0, step=0.001)

FloatSlider(value=0.0, description='steering offset', max=0.5, min=-0.5, step=0.001)

FloatSlider(value=0.0, description='speed gain', max=1.5, step=0.001)

FloatSlider(value=0.0, description='speed offset', max=1.0, min=-1.0, step=0.001)

Call the following cell to initialize the camera.

In [7]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_width=1280, capture_height=720, capture_fps=90)

camera.running = True

Now let's call our control loop function, which will be attached to new camera images.  This will execute the neural network and steer the car.

In [10]:
import traitlets

steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, value=0.0, step=0.001, description='steering slider')
speed_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.0, step=0.001, description='speed slider')

traitlets.dlink((car, 'throttle'), (speed_slider, 'value'))
traitlets.dlink((car, 'steering'), (steering_slider, 'value'))
display(steering_slider, speed_slider)

FloatSlider(value=0.0, description='steering slider', max=1.0, min=-1.0, step=0.001)

FloatSlider(value=0.0, description='speed slider', max=1.0, step=0.001)

In [45]:
from utils import preprocess

def execute(change):
    image = change['new']
    data = preprocess(image)
    xy_throttle = model_trt(data)[0].cpu()
    xy = apex_model_trt(data)[0].cpu()
    
    y_throttle = 0.5 - 0.5 *float(xy_throttle[1])
    
    x = float(xy[0])
    car.steering = x * steering_gain.value + steering_offset.value
    car.throttle = y_throttle * speed_gain.value + speed_offset.value

We'll manually call the execution function once, which may take some time the first time the neural network is called.

We recommend disabling autonomous mode for this first execution.

In [46]:
execute({'new': camera.value})

Now, assuming the cell above executed without issue, call the following cell to attach the execute function to new camera values.

In [47]:
camera.observe(execute, names='value')

In [48]:
camera.unobserve_all()